<a href="https://colab.research.google.com/github/YashNigam65/gitfolder/blob/master/similar_audio_search_with_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Write a python program to find similar audio files to the one you upload. Store many audio files and compare the one you upload with the stored ones. Using torchaudio framework or any other appropriate framework.

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.0 MB/s eta 

In [2]:
import librosa
import numpy as np
from chromadb.config import Settings
from chromadb import Client

In [5]:
# Step 1: Function to extract audio embeddings using Mel-frequency cepstral coefficients (MFCC)
def extract_audio_embedding(audio_path):
    y, sr = librosa.load(audio_path, sr=None)  # Load audio file #
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract MFCC features
    embedding = np.mean(mfcc, axis=1)  # Take the mean across time frames
    return embedding



```
# y, sr = librosa.load(audio_path, sr=None)
```



y: A NumPy array representing the audio time series (the audio waveform).

sr: The sampling rate of the audio. By setting sr=None, librosa will use the native sampling rate of the audio file.


In [6]:
# Step 2: Initialize ChromaDB client
client = Client(Settings())
collection = client.get_or_create_collection(name="audio_collection")

In [7]:
# Step 3: Add audio files to the vector database
def add_audio_to_database(audio_paths):
    for idx, audio_path in enumerate(audio_paths):
        embedding = extract_audio_embedding(audio_path)
        metadata = {"path": audio_path}
        collection.add(
            ids=[f"audio_{idx}"],  # Changed 'embedding_ids' to 'ids'
            embeddings=[embedding],
            metadatas=[metadata]
        )


In [13]:
# Step 4: Perform similarity search for an audio query
def search_similar_audio(query_audio_path, top_k=3):
    query_embedding = extract_audio_embedding(query_audio_path)
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    print("results", results)
    # Accessing the list of metadatas correctly:
    for result_metadata in results["metadatas"]:  # Iterate through the list of metadatas
        print("Similar Audio Found:", result_metadata[0]["path"])  # Access the path from the first element

In [15]:
# Step 5: Example usage
if __name__ == "__main__":
    # Example audio files (replace with your actual audio file paths)
    audio_files = [
        "/content/sample_data/yash_audio.m4a",  # Replace with real audio file paths
        "/content/sample_data/youTube.m4a",
        "/content/sample_data/boly_song.m4a",
        "/content/sample_data/dhun_yt.m4a"
    ]

    # Add audio files to the database
    add_audio_to_database(audio_files)

    # Query with a new audio file
    # query_audio = "/content/sample_data/yt.m4a"  # Replace with the query audio file path
    # search_similar_audio(query_audio, top_k=3)

    query_audio = "/content/sample_data/dhun_yt.m4a"  # Replace with the query audio file path
    search_similar_audio(query_audio, top_k=3)


/tmp/ipython-input-3251902400.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=None)  # Load audio file #
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-3251902400.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=None)  # Load audio file #
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-3251902400.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=None)  # 

results {'ids': [['audio_3', 'audio_2', 'audio_1']], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'path': '/content/sample_data/dhun_yt.m4a'}, {'path': '/content/sample_data/boly_song.m4a'}, {'path': '/content/sample_data/youTube.m4a'}]], 'distances': [[0.0, 2764.05322265625, 7687.98291015625]]}
Similar Audio Found: /content/sample_data/dhun_yt.m4a
